### 0. Load DataFrame from previous Jupyther Notebook!



In [ ]:
import pandas as pd
Canada = pd.read_pickle("./Canada.pkl")
Canada

### 1. Use the geocoder library to find coordinates for postalcodes

**Task:** *Use the Geocoder package to get the latitude and the longitude coordinates for all the neighborhoods in our dataframe.*

Define a function "find_coordinates", that returns the coordinates for a given Postalcode. I tried Google and some other geocoder apis. Sadly, sometimes a request to the api server ends with timeout, some postalcodes can't be resolved, or the count of the queries is limited. Therefore, at first I request "geocodefarm", and afterwards I call "Photon".

In [ ]:
#!conda install -c conda-forge geocoder --yes # install in case...

import geocoder
import numpy as np


def find_coordinates ( df ):
    # loop until found coords
    
    lat_lng_coords = None
    count = 0

    while(lat_lng_coords is None and count <10):
        lat_lng_coords = geocoder.geocodefarm( '{}, Canada'.format( df['Postalcode'] ))
        # Other API, I tried:
        #lat_lng_coords = Nominatim().geocode('{}, Canada'.format( df['Postalcode'] ) )
        #lat_lng_coords = geocoder.google('{}, Canada'.format(df['Postalcode'])).latlng
        #lat_lng_coords = geocoder.geocode( '{}, Canada'.format( df['Postalcode'] ))
        count +=1
    
    try:
        print ( 'Postcode: {}, Lat: {}, Lng: {}.'.format( 
            df['Postalcode'], 
            lat_lng_coords.json['lat'], 
            lat_lng_coords.json['lng']  ))
        return [ lat_lng_coords.longitude, lat_lng_coords.latitude ]
    
    except:
        print ( 'Postcode: {} is NA'.format( df['Postalcode'] ))
        return [ np.nan, np.nan ]
    
Canada[['Latitude','Longitude']] =  Canada.apply( find_coordinates, axis=1, result_type="expand")

If a postalcodes can't be resolved to their coordinates, I request those entries with Photon.

In [ ]:
from geopy.geocoders import Photon
geocoder = Photon()

print('Resolving postcalcodes...')

unlocatedCodes  = Canada['Postalcode'][ Canada['Latitude'].isna() ]
while not unlocatedCodes.empty:
    try:
        for code in unlocatedCodes.iloc[:]:
            print (  code )
            lat_lng_coords = geocoder.geocode( '{}, Canada'.format( code ))
            Canada.loc[ Canada['Postalcode'] == code, ['Latitude','Longitude']] = [ lat_lng_coords.longitude, lat_lng_coords.latitude ]
    except:
        pass
    finally:
        unlocatedCodes  = Canada['Postalcode'][ Canada['Latitude'].isna() ]

print ('All Postalcodes resolved to their coordinates: {}'.format( Canada['Postalcode'][ Canada['Latitude'].isna() ].empty ) )

Apply the function "find_coordinates" to each Postalcode in Canada and build new columns for these coordinates:

In [ ]:
Canada

### 2. Save DataFrame with coordinates

In [ ]:
Canada.to_pickle("./Canada_with_Coor.pkl")

In [ ]:
'''# For TEST purposes only: Geocoder API for Google
from pygeocoder import Geocoder
results = Geocoder.geocode("4207 N Washington Ave, Douglas, AZ 85607")
results.coordinates
'''